In [104]:
# Masterarbeit: F1 - Finale Analyse der Kompensationsforderungen

#Dieses Notebook dient der finalen Analyse der Umfragedaten zur Beantwortung der Forschungsfrage F1:
#_"Welche monetären Kompensationsforderungen stellen Haushalte in der Schweiz für die Teilnahme an einem Peak-Shaving-Programm unter Verwendung von Smart-Home-Technologien?"_

#Es folgt der überarbeiteten Struktur für den Ergebnisteil der Masterarbeit.

#**Struktur des Notebooks:**
#0. Setup und Datenladung (Erstellung des finalen `master_df_f1`)
#1. Deskriptive Statistik der Stichprobe
#2. Grundsätzliche Einstellungen zu erneuerbaren Energien und Netzstabilität (Kontext)
#3. Flexibilitätsbereitschaft der Haushalte (Akzeptanz von Lastverschiebung)
#    3.1 Wichtigkeit der Geräte-Verfügbarkeit (Q8)
#    3.2 Akzeptierte Nichtnutzungsdauer (Q9) pro Gerät
#    3.3 Bedingungen für die Teilnahme an Lastverschiebung (Q10 `incentive_choice`)
#4. Monetäre Kompensationsforderungen (Q10 `incentive_pct_required` - Kern von F1)
#5. Einflussfaktoren auf Teilnahmebereitschaft und Kompensationsforderungen
#    5.1 Soziodemografische Faktoren
#    5.2 Einstellungs- und kontextbezogene Faktoren
#6. (Optional) Synthese: Modelliertes Flexibilitätspotenzial (3D-Plots)
#7. Zusammenfassung der Kernergebnisse für F1 und Export

In [105]:
# === Abschnitt 0: Setup und Laden der Kerndaten ===
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
# import matplotlib.pyplot as plt # Alternative für manche Plots, falls benötigt
# import seaborn as sns          # Alternative für manche Plots, falls benötigt
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta
from src.logic.respondent_level_model.flexibility_analyzer import get_flexibility_potential

# --- Pfad-Setup ---
# Bitte stelle sicher, dass dieser Teil für deine lokale Ordnerstruktur korrekt funktioniert.
try:
    # Wenn als Skript ausgeführt (weniger wahrscheinlich für .ipynb)
    NOTEBOOK_FILE_PATH = Path(__file__).resolve()
    SCRIPT_DIR_F1 = NOTEBOOK_FILE_PATH.parent
except NameError:
    # Tritt auf, wenn __file__ nicht definiert ist (typisch für interaktive Notebook-Ausführung)
    SCRIPT_DIR_F1 = Path(os.getcwd()).resolve()
    print(f"Hinweis: __file__ nicht verfügbar, SCRIPT_DIR_F1 auf aktuelles Arbeitsverzeichnis gesetzt: {SCRIPT_DIR_F1}")

# Annahme: Dieses Notebook liegt in PowerE/scripts/F1_Analyse_Kompensationsforderungen/
# oder einem ähnlichen Pfad wie dein vorheriges Notebook.
# Passe dies ggf. an, falls du das neue Notebook woanders speicherst.
if SCRIPT_DIR_F1.name == "F1_Analyse_Kompensationsforderungen" and SCRIPT_DIR_F1.parent.name == "scripts" and SCRIPT_DIR_F1.parent.parent.name == "PowerE":
    PROJECT_ROOT_NB = SCRIPT_DIR_F1.parent.parent
elif SCRIPT_DIR_F1.name == "scripts" and SCRIPT_DIR_F1.parent.name == "PowerE": # Falls das Notebook direkt in 'scripts' liegt
     PROJECT_ROOT_NB = SCRIPT_DIR_F1.parent
elif SCRIPT_DIR_F1.name == "PowerE": # Falls das Notebook im Projekt-Root liegt
    PROJECT_ROOT_NB = SCRIPT_DIR_F1
else:
    # Fallback, versuche einen sinnvollen Projekt-Root zu finden oder setze ihn manuell.
    # Im Zweifel: PROJECT_ROOT_NB = Path("DEIN/PFAD/ZU/PowerE").resolve()
    current_path = Path(os.getcwd()).resolve()
    if current_path.name == "PowerE": # Sehr häufig, dass man im Projekt-Root startet
        PROJECT_ROOT_NB = current_path
    else: # Generischer Fallback: Gehe zwei Ebenen hoch vom aktuellen Notebook-Verzeichnis
        PROJECT_ROOT_NB = SCRIPT_DIR_F1.parent.parent
        print(f"WARNUNG: Projekt-Root-Bestimmung ist möglicherweise ungenau. PROJECT_ROOT_NB geraten als: {PROJECT_ROOT_NB}")
        print("Bitte stelle sicher, dass dies dein 'PowerE'-Verzeichnis ist und deine 'src'-Module von hier aus erreichbar sind.")

if str(PROJECT_ROOT_NB) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT_NB))
    print(f"Projekt-Root '{PROJECT_ROOT_NB}' zum sys.path hinzugefügt.")
else:
    print(f"Projekt-Root '{PROJECT_ROOT_NB}' ist bereits im sys.path.")

print(f"Verwendeter Projekt-Root für dieses Notebook: {PROJECT_ROOT_NB}")

# --- Importe deiner benutzerdefinierten Module ---
# Stelle sicher, dass diese Pfade relativ zum PROJECT_ROOT_NB korrekt sind.
try:
    from src.logic.respondent_level_model.data_transformer import create_respondent_flexibility_df
    from src.data_loader.survey_loader.demographics import load_demographics
    from src.data_loader.survey_loader.attitudes import load_attitudes
    from src.data_loader.survey_loader.demand_response import load_importance, load_notification, load_smart_plug
    from src.data_loader.survey_loader.socioeconomics import load_socioeconomics
    print("Alle benötigten Module aus 'src' erfolgreich importiert.")
except ImportError as e:
    print(f"FEHLER beim Importieren von Modulen aus 'src': {e}")
    print("Bitte stelle sicher, dass der PROJECT_ROOT_NB korrekt ist, alle __init__.py Dateien in 'src' und dessen Unterordnern vorhanden sind und die Modulnamen exakt stimmen.")
    # Hier könntest du sys.exit(1) verwenden, wenn das Notebook als Skript laufen würde,
    # aber im interaktiven Modus ist es besser, den Fehler anzuzeigen und ggf. manuell zu korrigieren.
    raise e # Fehler erneut auslösen, um die Zelle zu stoppen
except Exception as e:
    print(f"Ein unerwarteter Fehler beim Importieren deiner Module ist aufgetreten: {e}")
    raise e

# --- Lade df_respondent_flexibility (Kombination aus Q9 Dauer & Q10 Anreiz) ---
print("\nLade df_respondent_flexibility (Q9 & Q10 kombiniert)...")
df_flex = pd.DataFrame() # Initialisieren für den Fall eines Fehlers
try:
    # Die Funktion create_respondent_flexibility_df sollte intern das PROJECT_ROOT_NB
    # nicht mehr benötigen, wenn deine Ladefunktionen (load_q9_nonuse_long, etc.)
    # ihre Pfade relativ zum src-Ordner oder absolut korrekt bilden.
    # Falls sie es doch benötigen, müsstest du es übergeben: create_respondent_flexibility_df(PROJECT_ROOT_NB)
    df_flex = create_respondent_flexibility_df()
    print(f"df_flexibility geladen. Shape: {df_flex.shape}")
    if df_flex.empty:
        print("WARNUNG: df_flexibility ist leer! Die weitere Analyse wird nicht aussagekräftig sein.")
    else:
        # Stelle sicher, dass respondent_id als string behandelt wird für merges
        if 'respondent_id' in df_flex.columns:
            df_flex['respondent_id'] = df_flex['respondent_id'].astype(str)
        else:
            print("WARNUNG: 'respondent_id' Spalte nicht in df_flex gefunden.")

        # Wichtig: Umbenennung der Spalte für Klarheit und Konsistenz mit Q9 Mapping
        # Das q9_duration_mapping in create_respondent_flexibility_df erstellt 'max_duration_hours'.
        # Wir nennen sie hier explizit 'max_duration_hours_num' für numerische Analysen.
        if 'max_duration_hours' in df_flex.columns:
            df_flex.rename(columns={'max_duration_hours': 'max_duration_hours_num'}, inplace=True)
            df_flex['max_duration_hours_num'] = pd.to_numeric(df_flex['max_duration_hours_num'], errors='coerce')
            print("Spalte 'max_duration_hours' zu 'max_duration_hours_num' umbenannt und in numerischen Typ konvertiert.")
        else:
            print("WARNUNG: 'max_duration_hours' (oder 'max_duration_hours_num') nicht in df_flex.")

        # Ggf. incentive_pct_required auch numerisch machen, falls nicht schon in create_respondent_flexibility_df passiert
        if 'incentive_pct_required' in df_flex.columns:
            df_flex['incentive_pct_required_num'] = pd.to_numeric(df_flex['incentive_pct_required'], errors='coerce')
            # Optional: df_flex.drop(columns=['incentive_pct_required'], inplace=True) wenn die _num Spalte reicht
            print("Spalte 'incentive_pct_required' zu 'incentive_pct_required_num' konvertiert/erstellt.")
        else:
            print("WARNUNG: 'incentive_pct_required' nicht in df_flex.")

        print("\nErste Zeilen von df_flex (nach möglicher Umbenennung/Konvertierung):")
        # display(df_flex.head()) # In .py-Skripten ist display nicht verfügbar, print verwenden
        print(df_flex.head())
        print("\nInfo zu df_flex:")
        df_flex.info()

except FileNotFoundError as e:
    print(f"FEHLER beim Erstellen von df_flexibility (zugrundeliegende Q9/Q10 CSVs nicht gefunden?): {e}")
except Exception as e:
    print(f"Ein anderer Fehler beim Erstellen von df_flexibility: {e}")
    # raise e # Optional: Fehler weiterleiten, um Ausführung zu stoppen

# --- Laden und Integration weiterer relevanter Umfragedaten zu master_df_f1 ---
master_df_f1 = pd.DataFrame() # Initialisieren

if not df_flex.empty and 'respondent_id' in df_flex.columns:
    master_df_f1 = df_flex.copy()

    # Dictionary deiner Ladefunktionen und der Schlüssel, unter denen sie im master_df landen sollen
    # Das Format ist: "daten_gruppe": (lade_funktion, optional_präfix_falls_dict_zurückgegeben_wird)
    # Wenn die Ladefunktion direkt ein DataFrame zurückgibt, ist der zweite Teil None oder ein Key für den Merge
    data_loader_functions = {
        "demographics": (load_demographics, None),
        "socioeconomics": (load_socioeconomics, None),
        "attitudes": (load_attitudes, None), # Enthält Q6 (challenge_text) und Q7 (consequence)
        "q8_importance": (load_importance, "importance_rating"), # load_importance gibt DF zurück, Spalte für Wert ist 'importance_rating'
        "q11_notification": (load_notification, "q11_notify"), # Spalte für Wert ist 'q11_notify'
        "q12_smartplug": (load_smart_plug, "q12_smartplug") # Spalte für Wert ist 'q12_smartplug'
    }

    print("\n--- Beginne Laden und Mergen zusätzlicher Daten ---")
    for group_key, (loader_func, value_col_or_prefix) in data_loader_functions.items():
        print(f"\nLade und merge Daten für: {group_key}...")
        try:
            # Annahme: Deine Ladefunktionen erwarten PROJECT_ROOT_NB oder finden ihre Daten relativ.
            # Falls sie es explizit brauchen: loaded_data = loader_func(PROJECT_ROOT_NB)
            loaded_data = loader_func(PROJECT_ROOT_NB)

            if isinstance(loaded_data, dict): # Für Loader, die ein Dict von DFs zurückgeben
                for sub_key, df_to_merge in loaded_data.items():
                    print(f"  Merging Untergruppe: {sub_key} (Shape: {df_to_merge.shape if not df_to_merge.empty else 'leer'})")
                    if not df_to_merge.empty and 'respondent_id' in df_to_merge.columns:
                        df_to_merge['respondent_id'] = df_to_merge['respondent_id'].astype(str)
                        master_df_f1 = pd.merge(master_df_f1, df_to_merge, on="respondent_id", how="left", suffixes=('', f'_{sub_key}_dup'))
                    else:
                        print(f"    WARNUNG: DataFrame für '{sub_key}' aus '{group_key}' ist leer oder hat keine 'respondent_id'.")

            elif isinstance(loaded_data, pd.DataFrame): # Für Loader, die direkt einen DataFrame zurückgeben
                df_to_merge = loaded_data
                print(f"  Merging Datenquelle: {group_key} (Shape: {df_to_merge.shape if not df_to_merge.empty else 'leer'})")
                
                if not df_to_merge.empty and 'respondent_id' in df_to_merge.columns:
                    df_to_merge['respondent_id'] = df_to_merge['respondent_id'].astype(str)

                    if group_key == "q8_importance":
                        # ----- BEGINN SPEZIFISCHE LOGIK NUR FÜR q8_importance -----
                        print(f"    Spezifische Behandlung für '{group_key}' (Wide-Format erwartet).")
                        device_columns_q8 = [col for col in df_to_merge.columns if col != 'respondent_id']

                        if not device_columns_q8:
                            print(f"    WARNUNG: Keine Geräte-Spalten in '{group_key}' für melt-Operation gefunden (außer respondent_id). Überspringe Merge.")
                        else:
                            # value_col_or_prefix ist 'importance_rating' für q8_importance
                            df_q8_long = df_to_merge.melt(
                                id_vars=['respondent_id'],
                                value_vars=device_columns_q8,
                                var_name='device', 
                                value_name=value_col_or_prefix # Sollte 'importance_rating' sein
                            )
                            print(f"    '{group_key}' nach melt: Shape {df_q8_long.shape}. Erste Zeilen:")
                            print(df_q8_long.head(2)) # Nur eine kleine Vorschau
                            
                            master_df_f1 = pd.merge(master_df_f1, df_q8_long, on=["respondent_id", "device"], how="left", suffixes=('', f'_{group_key}_dup'))
                            print(f"    '{group_key}' (nach melt und merge) gemerged.")
                        # ----- ENDE SPEZIFISCHE LOGIK NUR FÜR q8_importance -----
                    
                    else: 
                        # ----- BEGINN STANDARD-MERGE FÜR ANDERE DATAFRAMES (z.B. Q11, Q12) -----
                        # Diese werden pro Respondent nur einmal gemerged und die Werte auf alle seine Gerätezeilen im master_df_f1 dupliziert,
                        # da master_df_f1 (basierend auf df_flex) mehrere Zeilen pro respondent_id (eine pro Gerät) hat.
                        print(f"    Standard-Merge für DataFrame '{group_key}' auf 'respondent_id'.")
                        master_df_f1 = pd.merge(master_df_f1, df_to_merge, on="respondent_id", how="left", suffixes=('', f'_{group_key}_dup'))
                        print(f"    '{group_key}' gemerged.")
                        # ----- ENDE STANDARD-MERGE FÜR ANDERE DATAFRAMES -----

                else: # Gehört zu: if not df_to_merge.empty and 'respondent_id' in df_to_merge.columns:
                    print(f"    WARNUNG: DataFrame für '{group_key}' ist leer oder hat keine 'respondent_id'. Wird übersprungen.")
            
            else:
                print(f"    WARNUNG: Unerwarteter Rückgabetyp von loader_func für '{group_key}'. Erwartet dict oder DataFrame, bekam {type(loaded_data)}.")

        except FileNotFoundError as e:
            print(f"    FEHLER bei {group_key}: Mindestens eine Datei nicht gefunden: {e}")
        except Exception as e:
            print(f"    Ein anderer Fehler beim Laden/Mergen von '{group_key}': {e}")
            import traceback
            traceback.print_exc()

    print("Spalten in master_df_f1:", master_df_f1.columns.tolist())
    print("\nZufällige Zeilen aus master_df_f1:")
    if not master_df_f1.empty:

    # display(master_df_f1.sample(5)) # Für Jupyter Notebook
        print(master_df_f1.sample(5)) # Für .py oder wenn display nicht geht
        # Überprüfe, ob Spalten wie 'age', 'q13_income', 'importance_rating', 'q11_notify' etc. Werte enthalten.
    else:
        print("master_df_f1 ist noch leer.")

        # Wichtig für die folgende deskriptive Statistik der Stichprobe (N=155)
    # Wir brauchen einen DataFrame mit unique Respondents für die Demografie etc.
    if 'respondent_id' in master_df_f1.columns:
        df_respondents_unique = master_df_f1.drop_duplicates(subset=['respondent_id']).copy()
        print(f"\nShape von df_respondents_unique (für Stichprobenbeschreibung): {df_respondents_unique.shape}") # Sollte (155, Anzahl_Spalten) sein

        # Sicherstellen, dass Spalten, die numerisch sein sollten, es auch sind
        if 'age' in master_df_f1.columns: # aus demographics
             master_df_f1['age'] = pd.to_numeric(master_df_f1['age'], errors='coerce')
        if 'importance_rating' in master_df_f1.columns: # aus q8_importance
             master_df_f1['importance_rating'] = pd.to_numeric(master_df_f1['importance_rating'], errors='coerce')

        # Überprüfe auf doppelte Spaltennamen nach dem Merge (außer 'respondent_id', 'device')
        cols = pd.Series(master_df_f1.columns)
        for dup in cols[cols.duplicated()].unique():
            print(f"WARNUNG: Doppelte Spalte '{dup}' im master_df_f1 nach Merge.")
            # Überlege dir eine Strategie, um Duplikate zu behandeln, z.B. eine bevorzugen oder umbenennen.
            # Beispiel: master_df_f1[dup] = master_df_f1[dup].bfill(axis=1).iloc[:,0] # Füllt NaN von links und nimmt die erste Spalte

        print("Erste 5 Zeilen von master_df_f1:")
        # display(master_df_f1.head())
        print(master_df_f1.head())
        print("\nInfo zu master_df_f1:")
        master_df_f1.info(verbose=True, show_counts=True)
        print(f"\nAnzahl unique Respondent IDs im master_df_f1: {master_df_f1['respondent_id'].nunique() if 'respondent_id' in master_df_f1.columns else 'N/A'}")
        print(f"Anzahl Zeilen im master_df_f1: {len(master_df_f1)}")
    else:
        print("WARNUNG: 'respondent_id' nicht in master_df_f1. df_respondents_unique kann nicht erstellt werden.")
        df_respondents_unique = pd.DataFrame()

else:
    print("FEHLER: df_flex ist leer oder hat keine 'respondent_id'-Spalte. Der Merge-Prozess für master_df_f1 wird übersprungen.")
    master_df_f1 = pd.DataFrame() # Sicherstellen, dass es existiert, auch wenn leer

print("\n--- Abschnitt 0: Setup und Datenladung Abgeschlossen ---")

Hinweis: __file__ nicht verfügbar, SCRIPT_DIR_F1 auf aktuelles Arbeitsverzeichnis gesetzt: /Users/jonathan/Documents/GitHub/PowerE/scripts/F1_Analyse_Kompensationsforderungen
Projekt-Root '/Users/jonathan/Documents/GitHub/PowerE' ist bereits im sys.path.
Verwendeter Projekt-Root für dieses Notebook: /Users/jonathan/Documents/GitHub/PowerE
Alle benötigten Module aus 'src' erfolgreich importiert.

Lade df_respondent_flexibility (Q9 & Q10 kombiniert)...
[INFO] create_respondent_flexibility_df: Starte Transformation der Umfragedaten...
[INFO] load_q9_nonuse_long: Überspringe die erste Datenzeile (Index 0), da sie wie ein Sub-Header aussieht.
[INFO] load_q9_nonuse_long: 775 Zeilen im langen Format aus Frage 9 geladen (nach Bereinigung).
  Q9-Daten verarbeitet (oder leer initialisiert): 775 Zeilen.
[INFO] load_q10_incentives_long: 775 Zeilen im langen Format aus Frage 10 geladen.
  Q10-Daten verarbeitet (oder leer initialisiert): 775 Zeilen.
  Q9 und Q10 Daten gemerged. Ergebnis-Shape vor fi

In [106]:
# Stelle sicher, dass df_respondents_unique aus dem vorherigen Code-Schnipsel existiert und nicht leer ist.
if 'df_respondents_unique' in locals() and not df_respondents_unique.empty:
    print(f"Beginne deskriptive Analyse der Stichprobe (N={len(df_respondents_unique)}).")

    # --- Q1: Alter ---
    # 'age' sollte bereits numerisch sein aus Abschnitt 0
    if 'age' in df_respondents_unique.columns:
        print("\n--- Q1: Alter ---")
        print("Deskriptive Statistik für Alter:")
        # display(df_respondents_unique['age'].describe())
        print(df_respondents_unique['age'].describe())
        
        fig_age_hist = px.histogram(df_respondents_unique.dropna(subset=['age']), x='age', nbins=10, title='Altersverteilung der Teilnehmer')
        fig_age_hist.update_layout(bargap=0.1)
        fig_age_hist.show()
        
        # Optional: Altersgruppen bilden für spätere Analysen oder tabellarische Darstellung
        # age_bins = [0, 18, 25, 35, 45, 55, 65, 120]
        # age_labels = ["<18", "18-25", "26-35", "36-45", "46-55", "56-65", "65+"]
        # df_respondents_unique['age_group'] = pd.cut(df_respondents_unique['age'], bins=age_bins, labels=age_labels, right=False)
        # print("\nVerteilung der Altersgruppen:")
        # print(df_respondents_unique['age_group'].value_counts(normalize=True).mul(100).round(1).sort_index())
    else:
        print("WARNUNG: Spalte 'age' nicht in df_respondents_unique gefunden.")

    # --- Q2: Geschlecht ---
    if 'gender' in df_respondents_unique.columns:
        print("\n--- Q2: Geschlecht ---")
        gender_counts = df_respondents_unique['gender'].value_counts(normalize=True).mul(100).round(1)
        # display(gender_counts.to_frame(name='Anteil (%)'))
        print(gender_counts.to_frame(name='Anteil (%)'))
        fig_gender = px.bar(gender_counts.reset_index(), x='gender', y='proportion', title='Geschlechterverteilung', text_auto=True)
        fig_gender.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_gender.show()
    else:
        print("WARNUNG: Spalte 'gender' nicht in df_respondents_unique gefunden.")

    # --- Q3: Haushaltsgrösse ---
    # 'household_size' muss ggf. noch in einen geordneten kategorischen oder numerischen Typ umgewandelt werden für eine sinnvolle Sortierung.
    if 'household_size' in df_respondents_unique.columns:
        print("\n--- Q3: Haushaltsgrösse ---")
        # Ggf. Werte wie ">6" oder "Über 6" behandeln und sortieren
        household_counts = df_respondents_unique['household_size'].value_counts(normalize=True).mul(100).round(1).sort_index()
        # display(household_counts.to_frame(name='Anteil (%)'))
        print(household_counts.to_frame(name='Anteil (%)'))
        fig_household = px.bar(household_counts.reset_index(), x='household_size', y='proportion', title='Verteilung der Haushaltsgrösse', text_auto=True)
        fig_household.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_household.update_xaxes(categoryorder='array', categoryarray=sorted(df_respondents_unique['household_size'].dropna().unique())) # Versuch der Sortierung
        fig_household.show()
    else:
        print("WARNUNG: Spalte 'household_size' nicht in df_respondents_unique gefunden.")

    # --- Q4: Wohnform ---
    if 'accommodation_type' in df_respondents_unique.columns:
        print("\n--- Q4: Wohnform ---")
        accommodation_counts = df_respondents_unique['accommodation_type'].value_counts(normalize=True).mul(100).round(1)
        # display(accommodation_counts.to_frame(name='Anteil (%)'))
        print(accommodation_counts.to_frame(name='Anteil (%)'))
        fig_accommodation = px.bar(accommodation_counts.reset_index(), x='accommodation_type', y='proportion', title='Verteilung der Wohnform', text_auto=True)
        fig_accommodation.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_accommodation.show()
    else:
        print("WARNUNG: Spalte 'accommodation_type' nicht in df_respondents_unique gefunden.")

    # --- Q5: Strombezug ---
    if 'electricity_type' in df_respondents_unique.columns:
        print("\n--- Q5: Strombezug ---")
        electricity_counts = df_respondents_unique['electricity_type'].value_counts(normalize=True).mul(100).round(1)
        # display(electricity_counts.to_frame(name='Anteil (%)'))
        print(electricity_counts.to_frame(name='Anteil (%)'))
        fig_electricity = px.bar(electricity_counts.reset_index(), x='electricity_type', y='proportion', title='Verteilung des Strombezugs', text_auto=True)
        fig_electricity.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_electricity.update_xaxes(tickangle=30) # Für bessere Lesbarkeit langer Labels
        fig_electricity.show()
    else:
        print("WARNUNG: Spalte 'electricity_type' nicht in df_respondents_unique gefunden.")

    # --- Q13: Haushaltseinkommen ---
    # 'q13_income' sollte als geordnete Kategorie behandelt werden für korrekte Sortierung in Plots/Tabellen
    if 'q13_income' in df_respondents_unique.columns:
        print("\n--- Q13: Haushaltseinkommen ---")
        income_order = [
            "Unter 3.000 CHF", "3.000 - 5.000 CHF", "5.001 - 7.000 CHF",
            "7.001 - 10.000 CHF", "Über 10.000 CHF", "Keine Angabe"
        ]
        # df_respondents_unique['q13_income_cat'] = pd.Categorical(
        #     df_respondents_unique['q13_income'], categories=income_order, ordered=True
        # )
        # income_counts = df_respondents_unique['q13_income_cat'].value_counts(normalize=True).mul(100).round(1).sort_index()
        # Stattdessen direkt mit der Originalspalte arbeiten und Plotly die Sortierung überlassen:
        income_counts = df_respondents_unique['q13_income'].value_counts(normalize=True).mul(100).round(1)
        income_counts = income_counts.reindex(income_order).fillna(0) # Sortieren und fehlende Kategorien mit 0 füllen

        # display(income_counts.to_frame(name='Anteil (%)'))
        print(income_counts.to_frame(name='Anteil (%)'))
        fig_income = px.bar(income_counts.reset_index(), x='q13_income', y='proportion', title='Verteilung des monatl. Haushaltsnettoeinkommens', text_auto=True)
        fig_income.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_income.update_xaxes(categoryorder='array', categoryarray=income_order, tickangle=30)
        fig_income.show()
    else:
        print("WARNUNG: Spalte 'q13_income' nicht in df_respondents_unique gefunden.")

    # --- Q14: Bildungsabschluss ---
    # 'q14_education' könnte auch als geordnete Kategorie sinnvoll sein.
    if 'q14_education' in df_respondents_unique.columns:
        print("\n--- Q14: Höchster Bildungsabschluss ---")
        education_order = [ # Passe diese Reihenfolge ggf. an deine exakten Kategorien und gewünschte Logik an
            "Keine Schulbildung", "Grundschule", "Sekundarschule/Realschule",
            "Berufsausbildung/Lehre/Maturität", "Fachhochschule/Bachelor",
            "Universität/Master", "Promotion oder höher", "Keine Angabe"
        ]
        education_counts = df_respondents_unique['q14_education'].value_counts(normalize=True).mul(100).round(1)
        try: # Versuche zu sortieren, falls Kategorien in `education_order` existieren
            education_counts = education_counts.reindex(education_order).fillna(0)
        except KeyError:
            print("WARNUNG: Nicht alle Bildungskategorien aus `education_order` in den Daten gefunden. Unsortierte Ausgabe.")
            education_counts = education_counts.sort_index() # Fallback: alphabetisch sortieren

        # display(education_counts.to_frame(name='Anteil (%)'))
        print(education_counts.to_frame(name='Anteil (%)'))
        fig_education = px.bar(education_counts.reset_index(), x='q14_education', y='proportion', title='Verteilung des höchsten Bildungsabschlusses', text_auto=True)
        fig_education.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_education.update_xaxes(categoryorder='array', categoryarray=education_order, tickangle=30)
        fig_education.show()
    else:
        print("WARNUNG: Spalte 'q14_education' nicht in df_respondents_unique gefunden.")

    # --- Q15: Politische Präferenz (optional) ---
    if 'q15_party' in df_respondents_unique.columns:
        print("\n--- Q15: Politische Präferenz ---")
        party_counts = df_respondents_unique['q15_party'].value_counts(normalize=True).mul(100).round(1)
        # display(party_counts.to_frame(name='Anteil (%)'))
        print(party_counts.to_frame(name='Anteil (%)'))
        # Für Plot ggf. kleine Parteien zu "Andere" zusammenfassen oder nur Top N zeigen
        fig_party = px.bar(party_counts.reset_index(), x='q15_party', y='proportion', title='Verteilung der politischen Präferenz', text_auto=True)
        fig_party.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_party.update_xaxes(tickangle=30)
        fig_party.show()
    else:
        print("WARNUNG: Spalte 'q15_party' nicht in df_respondents_unique gefunden.")

else:
    print("FEHLER: df_respondents_unique ist nicht verfügbar oder leer. Deskriptive Analyse der Stichprobe kann nicht durchgeführt werden.")

print("\n--- Abschnitt 1: Deskriptive Statistik der Stichprobe Abgeschlossen ---")

Beginne deskriptive Analyse der Stichprobe (N=155).

--- Q1: Alter ---
Deskriptive Statistik für Alter:
count    150.000000
mean      39.506667
std       16.407457
min       18.000000
25%       26.000000
50%       34.500000
75%       55.750000
max       75.000000
Name: age, dtype: float64



--- Q2: Geschlecht ---
          Anteil (%)
gender              
Männlich        49.7
Weiblich        49.0
Divers           1.3



--- Q3: Haushaltsgrösse ---
                Anteil (%)
household_size            
1                     18.7
2                     32.3
3                     20.0
4                     17.4
5                      6.5
6                      1.9
über 6                 3.2



--- Q4: Wohnform ---
                    Anteil (%)
accommodation_type            
Wohnung (Miete)           53.5
Haus (Eigentum)           27.1
Wohnung (Eigentum)        12.3
Haus (Miete)               7.1



--- Q5: Strombezug ---
                                                    Anteil (%)
electricity_type                                              
Eine Mischung aus konventionellem Strom und Öko...        32.9
Konventionellen Strom (Kernenergie und fossilen...        30.3
Weiss nicht                                               18.7
Ökostrom (aus erneuerbaren Energien wie Wasser,...        18.1



--- Q13: Haushaltseinkommen ---
                    Anteil (%)
q13_income                    
Unter 3.000 CHF            9.0
3.000 - 5.000 CHF         15.5
5.001 - 7.000 CHF         14.8
7.001 - 10.000 CHF        20.0
Über 10.000 CHF           23.9
Keine Angabe              16.8



--- Q14: Höchster Bildungsabschluss ---
                                  Anteil (%)
q14_education                               
Keine Schulbildung                       1.9
Grundschule                              0.0
Sekundarschule/Realschule                5.8
Berufsausbildung/Lehre/Maturität        42.6
Fachhochschule/Bachelor                 21.9
Universität/Master                      25.2
Promotion oder höher                     1.3
Keine Angabe                             1.3



--- Q15: Politische Präferenz ---
                                             Anteil (%)
q15_party                                              
Ich habe keine feste Parteipräferenz               27.1
Keine Angabe                                       15.5
Sozialdemokratische Partei der Schweiz (SP)        12.3
Schweizerische Volkspartei (SVP)                   11.6
FDP.Die Liberalen (FDP)                            11.0
Die Mitte                                           8.4
Grüne Partei der Schweiz (GPS)                      5.8
Grünliberale Partei (GLP)                           5.8
Andere Partei                                       1.9
Evangelische Volkspartei (EVP)                      0.6



--- Abschnitt 1: Deskriptive Statistik der Stichprobe Abgeschlossen ---


In [107]:
# Abschnitt 3: Flexibilitätsbereitschaft der Haushalte (Akzeptanz von Lastverschiebung)
## 3.1 Wichtigkeit der Geräte-Verfügbarkeit (Q8)
# Stelle sicher, dass master_df_f1 aus Abschnitt 0 existiert und nicht leer ist.
# Für diese Analyse verwenden wir master_df_f1, da 'importance_rating' gerätespezifisch ist.
if 'master_df_f1' in locals() and not master_df_f1.empty:
    print(f"Beginne Analyse der Gerätewichtigkeit (Q8) basierend auf master_df_f1 (Shape: {master_df_f1.shape})")

    if 'device' in master_df_f1.columns and 'importance_rating' in master_df_f1.columns:
        print("\n--- Q8: Wichtigkeit der jederzeitigen Gerätenutzung ---")
        
        # Berechne deskriptive Statistiken (Mittelwert, Median, Std) pro Gerät
        # 'importance_rating' sollte bereits numerisch sein aus Abschnitt 0
        q8_stats_per_device = master_df_f1.groupby('device')['importance_rating'].agg(['mean', 'median', 'std', 'count']).round(2)
        q8_stats_per_device = q8_stats_per_device.sort_values(by='mean', ascending=False) # Sortieren nach mittlerer Wichtigkeit
        
        print("Mittlere und mediane Wichtigkeit pro Gerät (1=sehr unwichtig, 5=sehr wichtig):")
        # display(q8_stats_per_device)
        print(q8_stats_per_device)
        
        # Visualisierung der mittleren Wichtigkeit
        # Wir nehmen den Index (Gerätenamen) für die x-Achse und 'mean' für die y-Achse
        fig_q8_importance = px.bar(q8_stats_per_device.reset_index(), 
                                   x='device', y='mean',
                                   title='Q8: Mittlere Wichtigkeit der jederzeitigen Gerätenutzung',
                                   labels={'device': 'Haushaltsgerät', 'mean': 'Mittlere Wichtigkeit (1-5)'},
                                   text='mean') # Zeigt den Wert auf den Balken an
        fig_q8_importance.update_traces(texttemplate='%{text:.2f}', textposition='outside')
        fig_q8_importance.update_yaxes(range=[0, 5.5]) # Y-Achse von 0 bis 5.5 für Kontext
        fig_q8_importance.show()

        # Optional: Verteilung der Wichtigkeitsratings pro Gerät (Boxplots)
        # fig_q8_box = px.box(master_df_f1.dropna(subset=['importance_rating']), 
        #                     x='device', y='importance_rating',
        #                     title='Q8: Verteilung der Wichtigkeitsratings pro Gerät',
        #                     labels={'device': 'Haushaltsgerät', 'importance_rating': 'Wichtigkeit (1-5)'})
        # fig_q8_box.update_yaxes(categoryorder='array', categoryarray=sorted(master_df_f1['device'].dropna().unique()))
        # fig_q8_box.show()
        
    else:
        print("WARNUNG: Spalten 'device' oder 'importance_rating' (für Q8) nicht in master_df_f1 gefunden.")

else:
    print("FEHLER: master_df_f1 ist nicht verfügbar oder leer. Analyse der Gerätewichtigkeit (Q8) kann nicht durchgeführt werden.")

print("\n--- Abschnitt 3.1: Wichtigkeit der Geräte-Verfügbarkeit (Q8) Abgeschlossen ---")

Beginne Analyse der Gerätewichtigkeit (Q8) basierend auf master_df_f1 (Shape: (775, 19))

--- Q8: Wichtigkeit der jederzeitigen Gerätenutzung ---
Mittlere und mediane Wichtigkeit pro Gerät (1=sehr unwichtig, 5=sehr wichtig):
                                     mean  median   std  count
device                                                        
Backofen und Herd                    4.23     5.0  1.08    155
Bürogeräte                           3.89     4.0  1.27    154
Waschmaschine                        3.73     4.0  1.24    155
Fernseher und Entertainment-Systeme  3.34     3.0  1.35    155
Geschirrspüler                       2.90     3.0  1.35    155



--- Abschnitt 3.1: Wichtigkeit der Geräte-Verfügbarkeit (Q8) Abgeschlossen ---


In [108]:
## 3.2 Akzeptierte Nichtnutzungsdauer (Q9) pro Gerät
# Stelle sicher, dass master_df_f1 aus Abschnitt 0 existiert und nicht leer ist.
if 'master_df_f1' in locals() and not master_df_f1.empty:
    print(f"Beginne Analyse der akzeptierten Nichtnutzungsdauer (Q9) pro Gerät basierend auf master_df_f1 (Shape: {master_df_f1.shape})")

    if 'device' in master_df_f1.columns and 'max_duration_hours_num' in master_df_f1.columns:
        print("\n--- Q9: Verteilung der akzeptierten Nichtnutzungsdauer pro Gerät ---")

        # Gruppiere nach Gerät und dann nach der Dauer, zähle die Häufigkeiten und normalisiere pro Gerät
        q9_duration_per_device = master_df_f1.groupby('device')['max_duration_hours_num'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        q9_duration_per_device_table = q9_duration_per_device.unstack(level='max_duration_hours_num', fill_value=0)

        # Definiere die gewünschte Reihenfolge der Spalten (Dauer-Kategorien)
        # Es ist wichtig, auch NaN zu berücksichtigen, falls vorhanden, oder sicherzustellen, dass keine da sind.
        # Die Kategorien stammen aus deinem q9_duration_mapping.
        duration_categories_q9_order = [0.0, 1.5, 4.5, 9.0, 18.0, 30.0, np.nan]
        
        # Stelle sicher, dass alle Kategorien in den Spalten vorhanden sind, fülle fehlende mit 0
        for duration_cat in duration_categories_q9_order:
            if duration_cat not in q9_duration_per_device_table.columns:
                q9_duration_per_device_table[duration_cat] = 0
        
        # Sortiere die Spalten gemäß der definierten Reihenfolge
        # Filtere Spalten, die tatsächlich im DataFrame sind, um KeyErrors zu vermeiden, falls np.nan nicht als Spalte existiert
        existing_cols_in_order = [col for col in duration_categories_q9_order if col in q9_duration_per_device_table.columns]
        q9_duration_per_device_table = q9_duration_per_device_table[existing_cols_in_order]

        print("Prozentuale Verteilung der akzeptierten Nichtnutzungsdauer pro Gerät (%):")
        # display(q9_duration_per_device_table)
        print(q9_duration_per_device_table)

        # Für die Visualisierung mit Plotly Express ist ein langes Format oft einfacher.
        # Wir nehmen q9_duration_per_device (Series mit MultiIndex) und wandeln es in einen DataFrame um.
        q9_duration_per_device_long_df = q9_duration_per_device.rename('percentage').reset_index()
        
        # Stelle sicher, dass max_duration_hours_num als Kategorie behandelt wird für korrekte Plot-Sortierung
        # und benenne NaN um für bessere Darstellung im Plot
        q9_duration_per_device_long_df['max_duration_hours_cat'] = q9_duration_per_device_long_df['max_duration_hours_num'].fillna('Keine Angabe Q9')
        # Definiere die Reihenfolge für die x-Achse des Plots
        plot_category_order_q9 = [str(cat) for cat in [0.0, 1.5, 4.5, 9.0, 18.0, 30.0]] + ['Keine Angabe Q9']
        q9_duration_per_device_long_df['max_duration_hours_cat'] = pd.Categorical(
            q9_duration_per_device_long_df['max_duration_hours_cat'].astype(str), #astype(str) um np.nan sicher zu behandeln
            categories=plot_category_order_q9,
            ordered=True
        )


        fig_q9_duration_device = px.bar(q9_duration_per_device_long_df,
                                        x='device', y='percentage',
                                        color='max_duration_hours_cat',
                                        title='Q9: Akzeptierte Nichtnutzungsdauer pro Gerät',
                                        labels={'device': 'Haushaltsgerät', 
                                                'percentage': 'Anteil der Nennungen (%)',
                                                'max_duration_hours_cat': 'Max. Nichtnutzungsdauer (Stunden, aus Q9)'},
                                        barmode='group', # oder 'stack', je nach Präferenz
                                        text='percentage')
        fig_q9_duration_device.update_traces(texttemplate='%{text:.1f}%', textposition='outside', cliponaxis=False)
        fig_q9_duration_device.update_xaxes(categoryorder='array', categoryarray=sorted(master_df_f1['device'].dropna().unique()))
        fig_q9_duration_device.update_layout(legend_title_text='Max. Dauer (h)')
        fig_q9_duration_device.show()
        
    else:
        print("WARNUNG: Spalten 'device' oder 'max_duration_hours_num' (für Q9) nicht in master_df_f1 gefunden.")

else:
    print("FEHLER: master_df_f1 ist nicht verfügbar oder leer. Analyse der akzeptierten Nichtnutzungsdauer (Q9) pro Gerät kann nicht durchgeführt werden.")

print("\n--- Abschnitt 3.2: Akzeptierte Nichtnutzungsdauer (Q9) pro Gerät Abgeschlossen ---")

Beginne Analyse der akzeptierten Nichtnutzungsdauer (Q9) pro Gerät basierend auf master_df_f1 (Shape: (775, 19))

--- Q9: Verteilung der akzeptierten Nichtnutzungsdauer pro Gerät ---
Prozentuale Verteilung der akzeptierten Nichtnutzungsdauer pro Gerät (%):
max_duration_hours_num               0.0   1.5   4.5   9.0   18.0  30.0  NaN 
device                                                                       
Backofen und Herd                    22.6  16.1  16.1  20.0  18.1   7.1   0.0
Bürogeräte                           31.0  13.5  14.8  14.2  14.2  11.6   0.6
Fernseher und Entertainment-Systeme  18.1  12.3  11.6  10.3  21.9  25.8   0.0
Geschirrspüler                        5.8   7.1   8.4  17.4  25.2  36.1   0.0
Waschmaschine                         8.4   9.0   6.5  13.5  29.7  32.3   0.6



--- Abschnitt 3.2: Akzeptierte Nichtnutzungsdauer (Q9) pro Gerät Abgeschlossen ---


In [109]:
## 3.3 Bedingungen für die Teilnahme an Lastverschiebung (Q10 `incentive_choice`)
# Stelle sicher, dass master_df_f1 aus Abschnitt 0 existiert und nicht leer ist.
if 'master_df_f1' in locals() and not master_df_f1.empty:
    print(f"Beginne Analyse der Teilnahmebedingungen (Q10 incentive_choice) basierend auf master_df_f1 (Shape: {master_df_f1.shape})")

    # --- 3.3.1 Gesamtverteilung der Teilnahmebereitschaft (Q10) ---
    if 'incentive_choice' in master_df_f1.columns:
        print("\n--- 3.3.1 Gesamtverteilung der Teilnahmebereitschaft (Q10 `incentive_choice`) ---")
        overall_participation_q10 = master_df_f1['incentive_choice'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        
        # Umbenennen von NaN für bessere Darstellung, falls vorhanden
        overall_participation_q10.rename(index={np.nan: 'Keine Angabe Q10'}, inplace=True)
        
        print("Gesamtverteilung der Teilnahmebereitschaft (Q10) in %:")
        # display(overall_participation_q10.to_frame(name='Anteil (%)'))
        print(overall_participation_q10.to_frame(name='Anteil (%)'))
        
        fig_q10_overall = px.bar(overall_participation_q10.reset_index(), 
                                 x='incentive_choice', y='proportion', # 'proportion' ist Standardname von value_counts
                                 title='Gesamte Teilnahmebereitschaft (Q10)', 
                                 labels={'proportion':'Anteil der Nennungen (%)', 'incentive_choice': 'Teilnahme-Typ (Q10)'},
                                 text_auto=True)
        fig_q10_overall.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_q10_overall.show()
    else:
        print("WARNUNG: Spalte 'incentive_choice' nicht in master_df_f1 gefunden.")

    # --- 3.3.2 Teilnahmebereitschaft (Q10) pro Gerät ---
    if 'device' in master_df_f1.columns and 'incentive_choice' in master_df_f1.columns:
        print("\n--- 3.3.2 Teilnahmebereitschaft (Q10 `incentive_choice`) pro Gerät ---")
        q10_by_device = master_df_f1.groupby('device')['incentive_choice'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        q10_by_device_table = q10_by_device.unstack(level='incentive_choice', fill_value=0)
        
        # Umbenennen von NaN-Spalte für bessere Darstellung, falls vorhanden
        if np.nan in q10_by_device_table.columns:
            q10_by_device_table.rename(columns={np.nan: 'Keine Angabe Q10'}, inplace=True)

        print("Prozentuale Verteilung der Teilnahmebereitschaft (Q10) pro Gerät (%):")
        # display(q10_by_device_table)
        print(q10_by_device_table)

        q10_by_device_long_df = q10_by_device.rename('percentage').reset_index()
        # Umbenennen von NaN für bessere Darstellung im Plot
        q10_by_device_long_df['incentive_choice'] = q10_by_device_long_df['incentive_choice'].fillna('Keine Angabe Q10')
        
        fig_q10_device = px.bar(q10_by_device_long_df, x='device', y='percentage', color='incentive_choice',
                               barmode='group', title='Teilnahmebereitschaft (Q10) pro Gerät',
                               labels={'percentage':'Anteil der Nennungen (%)', 'device':'Haushaltsgerät', 'incentive_choice':'Teilnahme-Typ (Q10)'},
                               text_auto=True)
        fig_q10_device.update_traces(texttemplate='%{y:.1f}%', textposition='outside', cliponaxis=False)
        fig_q10_device.update_xaxes(categoryorder='array', categoryarray=sorted(master_df_f1['device'].dropna().unique()))
        fig_q10_device.show()
    else:
        print("WARNUNG: Spalten 'device' oder 'incentive_choice' nicht in master_df_f1 für Analyse pro Gerät gefunden.")

    # --- 3.3.3 Teilnahmebereitschaft (Q10) in Abhängigkeit der akzeptierten Nichtnutzungsdauer (Q9) ---
    if 'max_duration_hours_num' in master_df_f1.columns and 'incentive_choice' in master_df_f1.columns:
        print("\n--- 3.3.3 Teilnahmebereitschaft (Q10 `incentive_choice`) nach max. Nichtnutzungsdauer (Q9 `max_duration_hours_num`) ---")
        q10_by_duration_q9 = master_df_f1.groupby('max_duration_hours_num')['incentive_choice'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        q10_by_duration_q9_table = q10_by_duration_q9.unstack(level='incentive_choice', fill_value=0)

        # Umbenennen von NaN-Spalte für bessere Darstellung, falls vorhanden
        if np.nan in q10_by_duration_q9_table.columns:
            q10_by_duration_q9_table.rename(columns={np.nan: 'Keine Angabe Q10'}, inplace=True)
        
        # Sicherstellen, dass die Zeilen (Dauer-Kategorien) in gewünschter Reihenfolge sind
        duration_categories_q9_order_rows = [0.0, 1.5, 4.5, 9.0, 18.0, 30.0, np.nan]
        # Filtere Zeilen, die tatsächlich im DataFrame sind, um KeyErrors zu vermeiden
        existing_rows_in_order = [row for row in duration_categories_q9_order_rows if row in q10_by_duration_q9_table.index]
        if np.nan in duration_categories_q9_order_rows and np.nan not in existing_rows_in_order and q10_by_duration_q9_table.index.hasnans: # Falls NaN im Index aber nicht explizit da ist
            existing_rows_in_order.append(np.nan)

        q10_by_duration_q9_table = q10_by_duration_q9_table.reindex(existing_rows_in_order)


        print("Prozentuale Verteilung der Teilnahmebereitschaft (Q10) nach max. Nichtnutzungsdauer (Q9) (%):")
        # display(q10_by_duration_q9_table)
        print(q10_by_duration_q9_table)

        q10_by_duration_q9_long_df = q10_by_duration_q9.rename('percentage').reset_index()
        # Umbenennen von NaN für bessere Darstellung im Plot (sowohl für Dauer als auch für incentive_choice)
        q10_by_duration_q9_long_df['max_duration_hours_cat'] = q10_by_duration_q9_long_df['max_duration_hours_num'].fillna('Keine Angabe Q9')
        q10_by_duration_q9_long_df['incentive_choice'] = q10_by_duration_q9_long_df['incentive_choice'].fillna('Keine Angabe Q10')
        
        # Definiere die Reihenfolge für die x-Achse des Plots (Dauer)
        plot_duration_category_order_q9 = [str(cat) for cat in [0.0, 1.5, 4.5, 9.0, 18.0, 30.0]] + ['Keine Angabe Q9']
        q10_by_duration_q9_long_df['max_duration_hours_cat'] = pd.Categorical(
            q10_by_duration_q9_long_df['max_duration_hours_cat'].astype(str),
            categories=plot_duration_category_order_q9,
            ordered=True
        )
        
        fig_q10_duration_q9 = px.bar(q10_by_duration_q9_long_df.sort_values('max_duration_hours_cat'), # Sortiere nach Dauer für den Plot
                                     x='max_duration_hours_cat', y='percentage', color='incentive_choice',
                                     barmode='group', title='Teilnahmebereitschaft (Q10) nach max. Nichtnutzungsdauer (Q9)',
                                     labels={'percentage':'Anteil der Nennungen (%)', 
                                             'max_duration_hours_cat':'Max. Nichtnutzungsdauer (Stunden, aus Q9)', 
                                             'incentive_choice':'Teilnahme-Typ (Q10)'},
                                     text_auto=True)
        fig_q10_duration_q9.update_traces(texttemplate='%{y:.1f}%', textposition='outside', cliponaxis=False)
        fig_q10_duration_q9.show()
    else:
        print("WARNUNG: Spalten 'max_duration_hours_num' oder 'incentive_choice' nicht in master_df_f1 für Analyse nach Dauer gefunden.")

else:
    print("FEHLER: master_df_f1 ist nicht verfügbar oder leer. Analyse der Teilnahmebedingungen (Q10) kann nicht durchgeführt werden.")

print("\n--- Abschnitt 3.3: Bedingungen für die Teilnahme (Q10 `incentive_choice`) Abgeschlossen ---")

Beginne Analyse der Teilnahmebedingungen (Q10 incentive_choice) basierend auf master_df_f1 (Shape: (775, 19))

--- 3.3.1 Gesamtverteilung der Teilnahmebereitschaft (Q10 `incentive_choice`) ---
Gesamtverteilung der Teilnahmebereitschaft (Q10) in %:
                  Anteil (%)
incentive_choice            
no                      39.5
yes_conditional         30.1
yes_fixed               23.6
unknown_choice           6.8



--- 3.3.2 Teilnahmebereitschaft (Q10 `incentive_choice`) pro Gerät ---
Prozentuale Verteilung der Teilnahmebereitschaft (Q10) pro Gerät (%):
incentive_choice                       no  unknown_choice  yes_conditional  \
device                                                                       
Backofen und Herd                    54.8             7.1             27.7   
Bürogeräte                           50.3             5.8             21.3   
Fernseher und Entertainment-Systeme  30.3             7.1             33.5   
Geschirrspüler                       22.6             7.7             34.2   
Waschmaschine                        39.4             6.5             33.5   

incentive_choice                     yes_fixed  
device                                          
Backofen und Herd                         10.3  
Bürogeräte                                22.6  
Fernseher und Entertainment-Systeme       29.0  
Geschirrspüler                            35.5  
Waschmaschine    


--- 3.3.3 Teilnahmebereitschaft (Q10 `incentive_choice`) nach max. Nichtnutzungsdauer (Q9 `max_duration_hours_num`) ---
Prozentuale Verteilung der Teilnahmebereitschaft (Q10) nach max. Nichtnutzungsdauer (Q9) (%):
incentive_choice          no  unknown_choice  yes_conditional  yes_fixed
max_duration_hours_num                                                  
0.0                     72.2             6.8             16.5        4.5
1.5                     41.1             4.4             36.7       17.8
4.5                     40.4             6.7             32.6       20.2
9.0                     33.3             6.0             33.3       27.4
18.0                    35.5             8.3             29.0       27.2
30.0                    21.1             7.4             34.3       37.1



--- Abschnitt 3.3: Bedingungen für die Teilnahme (Q10 `incentive_choice`) Abgeschlossen ---


In [110]:
# Abschnitt 4: Monetäre Kompensationsforderungen und modelliertes Flexibilitätspotenzial
## 4.1 Deskriptive Analyse der explizit genannten Kompensationsforderungen pro Gerät
# Stelle sicher, dass master_df_f1 aus Abschnitt 0 existiert und nicht leer ist.
if 'master_df_f1' in locals() and not master_df_f1.empty:
    print(f"Beginne deskriptive Analyse der explizit genannten Kompensationsforderungen pro Gerät (basierend auf master_df_f1)")

    if 'incentive_choice' in master_df_f1.columns and 'incentive_pct_required_num' in master_df_f1.columns:
        # Erstelle DataFrame nur mit Teilnehmern, die Kompensation fordern und einen Wert angegeben haben
        df_compens_demand_stated = master_df_f1[master_df_f1['incentive_choice'] == 'yes_conditional'].copy()
        
        original_yes_conditional_count = len(df_compens_demand_stated)
        df_compens_demand_stated.dropna(subset=['incentive_pct_required_num'], inplace=True)
        valid_compens_demand_count = len(df_compens_demand_stated)
        
        print(f"Anzahl der Nennungen mit 'yes_conditional': {original_yes_conditional_count}")
        if original_yes_conditional_count > 0:
            print(f"Anzahl davon mit valider Kompensationsforderung (für diese deskriptive Analyse): {valid_compens_demand_count} ({(valid_compens_demand_count/original_yes_conditional_count*100):.1f}%)")
        else:
            print("Keine Nennungen mit 'yes_conditional' gefunden.")

        if not df_compens_demand_stated.empty:
            print("\n--- Deskriptive Statistik der explizit genannten Kompensationsforderung pro Gerät ---")
            # Berechnung der deskriptiven Statistiken pro Gerät
            # Die Spalte '50%' entspricht dem Median.
            stated_compens_stats_per_device = df_compens_demand_stated.groupby('device')['incentive_pct_required_num'].describe().round(2)
            
            print("Deskriptive Statistik der Kompensationsforderung pro Gerät (explizit genannt):")
            # display(stated_compens_stats_per_device)
            print(stated_compens_stats_per_device)
            
            # Optional: Einfacher Bar-Plot der Mediane (Spalte '50%') oder Mittelwerte ('mean')
            # Wir verwenden hier den Median, da er robuster gegenüber Ausreissern ist.
            # Sortierung für den Plot nach Median
            plot_data_median_compens = stated_compens_stats_per_device[['50%']].sort_values(by='50%', ascending=True).reset_index()

            fig_median_compens_device = px.bar(plot_data_median_compens, 
                                               x='device', y='50%',
                                               title='Median der explizit genannten Kompensationsforderung pro Gerät',
                                               labels={'device': 'Haushaltsgerät', '50%': 'Median Kompensation (%)'},
                                               text='50%')
            fig_median_compens_device.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
            fig_median_compens_device.update_yaxes(range=[0, plot_data_median_compens['50%'].max() * 1.15]) # Y-Achsenbereich anpassen
            fig_median_compens_device.update_xaxes(categoryorder='array', categoryarray=plot_data_median_compens['device']) # Reihenfolge beibehalten
            fig_median_compens_device.show()

        else:
            print("Keine Daten für 'yes_conditional' Teilnehmer mit validen expliziten Kompensationsforderungen gefunden.")
    else:
        print("WARNUNG: Spalten 'incentive_choice' oder 'incentive_pct_required_num' nicht in master_df_f1 gefunden.")
else:
    print("FEHLER: master_df_f1 ist nicht verfügbar oder leer. Analyse kann nicht durchgeführt werden.")

print("\n--- Abschnitt 4.1: Deskriptive Analyse der explizit genannten Kompensationsforderungen pro Gerät Abgeschlossen ---")

Beginne deskriptive Analyse der explizit genannten Kompensationsforderungen pro Gerät (basierend auf master_df_f1)
Anzahl der Nennungen mit 'yes_conditional': 233
Anzahl davon mit valider Kompensationsforderung (für diese deskriptive Analyse): 187 (80.3%)

--- Deskriptive Statistik der explizit genannten Kompensationsforderung pro Gerät ---
Deskriptive Statistik der Kompensationsforderung pro Gerät (explizit genannt):
                                     count   mean    std  min   25%   50%  \
device                                                                      
Backofen und Herd                     33.0  21.85  23.94  2.0  10.0  10.0   
Bürogeräte                            28.0  21.86  20.34  1.0  10.0  15.0   
Fernseher und Entertainment-Systeme   41.0  23.49  22.27  2.0  10.0  15.0   
Geschirrspüler                        45.0  20.16  18.60  2.0  10.0  10.0   
Waschmaschine                         40.0  26.32  22.86  4.0  10.0  20.0   

                                      


--- Abschnitt 4.1: Deskriptive Analyse der explizit genannten Kompensationsforderungen pro Gerät Abgeschlossen ---


In [111]:
# Stelle sicher, dass master_df_f1 und get_flexibility_potential existieren.
if 'master_df_f1' in locals() and not master_df_f1.empty and 'get_flexibility_potential' in globals():
    print(f"Beginne modellbasierte Analyse des Flexibilitätspotenzials (basierend auf master_df_f1)")

    # --- Simulationsparameter definieren (angelehnt an visualize_flexibility_surface.py) ---
    base_sim_assumptions = {
        'reality_discount_factor': 0.7, # Wie in deinem Skript
        'payback_model': {'type': 'none'} # Annahme für dieses Modell
    }
    
    # Anreizstufen (X-Achse)
    incentives_range = np.linspace(0, 50, 11) # 0, 5, 10, ..., 50 %
    print(f"Verwendete Anreizstufen (Kompensation %): {incentives_range}")

    # Dauerstufen (Y-Achse) - Logik aus visualize_flexibility_surface.py adaptiert
    # Diese Logik bestimmt die zu testenden Dauern basierend auf den Antworten in Q9
    # und einer maximal gewünschten Dauer für die Plots.
    MAX_DURATION_FOR_PLOTS_Q9 = 18.0 # Maximale Dauer, die wir aus Q9 für die Y-Achse berücksichtigen wollen

    durations_range_final_q9 = np.array([0.5, 1.0, 1.5, 2.0, 2.5, 3.0]) # Standard-Fallback
    if 'max_duration_hours_num' in master_df_f1.columns:
        unique_survey_durations = sorted(master_df_f1['max_duration_hours_num'].dropna().unique())
        # Filtere Dauern, die > 0 und <= MAX_DURATION_FOR_PLOTS_Q9 sind
        durations_range_filtered = [d for d in unique_survey_durations if 0 < d <= MAX_DURATION_FOR_PLOTS_Q9]
        
        if not durations_range_filtered:
            all_positive_survey_durations = [d for d in unique_survey_durations if d > 0]
            if all_positive_survey_durations:
                temp_range = [d for d in all_positive_survey_durations if d <= MAX_DURATION_FOR_PLOTS_Q9]
                if not temp_range: 
                    durations_range_final_q9 = np.array([min(all_positive_survey_durations)])
                else:
                    durations_range_final_q9 = np.array(sorted(list(set(temp_range))))
            else: 
                durations_range_final_q9 = np.array([d for d in [0.5, 1.0, 1.5] if d <= MAX_DURATION_FOR_PLOTS_Q9 and d > 0])
                if durations_range_final_q9.size == 0: durations_range_final_q9 = np.array([0.5])
        else:
            durations_range_final_q9 = np.array(sorted(list(set(durations_range_filtered))))
    else:
        print("WARNUNG: 'max_duration_hours_num' nicht in master_df_f1. Verwende Standard-Range für Dauer.")
        durations_range_final_q9 = np.array([d for d in durations_range_final_q9 if d <= MAX_DURATION_FOR_PLOTS_Q9 and d > 0])
        if durations_range_final_q9.size == 0 and MAX_DURATION_FOR_PLOTS_Q9 > 0 : durations_range_final_q9 = np.array([min(0.5, MAX_DURATION_FOR_PLOTS_Q9)])
        elif durations_range_final_q9.size == 0 : durations_range_final_q9 = np.array([0.5])

    if durations_range_final_q9.size == 0:
        print(f"FEHLER: `durations_range_final_q9` ist leer. Plot kann nicht erstellt werden.")
        # Hier ggf. eine Exception werfen oder anders behandeln
    else:
        print(f"Verwendete Event-Dauern für die Simulation (basierend auf Q9, max {MAX_DURATION_FOR_PLOTS_Q9}h): {durations_range_final_q9} Stunden")

        # --- Daten für ein Zielgerät generieren (z.B. Geschirrspüler) ---
        target_appliance_surface = "Geschirrspüler" # Wir starten mit dem "Star"
        print(f"\nGeneriere Daten für 3D-Plot für Zielgerät: {target_appliance_surface}...")

        # Erstellung eines repräsentativen (konstanten) Lastprofils für das Gerät
        # Dauer des Profils muss die längste zu simulierende Dauer abdecken
        max_profile_duration_h = durations_range_final_q9.max() if durations_range_final_q9.size > 0 else MAX_DURATION_FOR_PLOTS_Q9
        num_intervals_profile = int(np.ceil(max_profile_duration_h / 0.25)) # 15-Minuten-Intervalle
        if num_intervals_profile == 0: num_intervals_profile = 1 # Mindestens ein Intervall
        
        dummy_start_time_profile = datetime(2024, 1, 1, 12, 0, 0) # Dummy-Startzeit
        plot_profile_index = pd.date_range(start=dummy_start_time_profile, periods=num_intervals_profile, freq="15T")
        
        # Annahme typischer Leistungen (kannst du anpassen oder aus anderen Quellen beziehen)
        power_kw_typical = {"Geschirrspüler": 1.2, "Waschmaschine": 1.0, "Backofen und Herd": 2.0, 
                              "Fernseher und Entertainment-Systeme": 0.2, "Bürogeräte": 0.3}
        appliance_power = power_kw_typical.get(target_appliance_surface, 1.0) # Default 1kW

        representative_profile_one_appliance = pd.DataFrame(
            {target_appliance_surface: appliance_power}, index=plot_profile_index
        )
        print(f"Verwendetes repräsentatives Profil für '{target_appliance_surface}': Konstant {appliance_power} kW über {max_profile_duration_h}h.")

        # Meshgrid für X (Anreize) und Y (Dauern)
        X_incentives, Y_durations = np.meshgrid(incentives_range, durations_range_final_q9)
        Z_participation_rate = np.zeros(X_incentives.shape)
        Z_shifted_energy_kwh = np.zeros(X_incentives.shape) # Optional, falls wir auch die Energie betrachten wollen

        # Berechnung der Z-Werte (Teilnahmequote)
        for i, duration_val in enumerate(durations_range_final_q9):
            print(f"  Bearbeite Dauer: {duration_val:.1f}h für {target_appliance_surface}...")
            for j, incentive_val in enumerate(incentives_range):
                # Sicherstellen, dass das Dummy-Profil das Event abdeckt
                event_end_time = dummy_start_time_profile + timedelta(hours=duration_val)
                profile_end_time = representative_profile_one_appliance.index.max() + timedelta(minutes=15) # Ende des letzten Intervalls

                if event_end_time > profile_end_time:
                    print(f"    WARNUNG: Event-Dauer {duration_val}h überschreitet Profillänge für {target_appliance_surface}. Setze Metriken auf NaN.")
                    Z_participation_rate[i, j] = np.nan
                    Z_shifted_energy_kwh[i, j] = np.nan
                    continue

                metrics = get_flexibility_potential(
                    appliance_name=target_appliance_surface,
                    event_duration_hours=duration_val,
                    incentive_pct=incentive_val,
                    df_respondent_flexibility=master_df_f1, # Unser Haupt-DataFrame mit Q9/Q10-Antworten
                    df_average_load_profile_appliance_only=representative_profile_one_appliance.copy(),
                    base_simulation_assumptions=base_sim_assumptions,
                    dummy_event_start_time=dummy_start_time_profile 
                )
                Z_participation_rate[i, j] = metrics["participation_rate"] * 100 # Direkt in Prozent für den Plot
                Z_shifted_energy_kwh[i, j] = metrics["shifted_energy_kwh"]
        
        print(f"Daten für {target_appliance_surface} generiert.")

        # --- 3D-Visualisierung für das Zielgerät ---
        if Z_participation_rate.size > 0 and not np.all(np.isnan(Z_participation_rate)):
            print(f"Erstelle 3D-Plot für Teilnahmequote: {target_appliance_surface}...")
            fig_3d_participation = go.Figure(data=[go.Surface(
                z=Z_participation_rate, x=X_incentives, y=Y_durations, 
                colorscale='Viridis', # Wähle eine passende Farbskala
                colorbar_title='Teilnahme (%)', 
                name=target_appliance_surface,
                contours = {"z": {"show": True, "highlightcolor":"limegreen", "project":{"z": True}}}
            )])
            fig_3d_participation.update_layout(
                title=f'Flexibilitätspotenzial: {target_appliance_surface}<br>Modellierte Teilnahmequote (Personen-basiert, %)',
                scene=dict(
                    xaxis_title='Anreiz (Kompensation in %)',
                    yaxis_title='Event-Dauer (Stunden, aus Q9)',
                    zaxis_title='Teilnahmequote (%)',
                    camera=dict(eye=dict(x=1.9, y=-1.2, z=1.2)), # Kameraposition anpassen
                    aspectmode='cube' 
                ),
                margin=dict(l=10, r=10, b=10, t=80)
            )
            fig_3d_participation.show()
            
            # Hier könnten wir auch spezifische Werte aus Z_participation_rate extrahieren und anzeigen
            # Z.B. Teilnahme bei 15% Anreiz für verschiedene Dauern:
            # incentive_index_15pct = np.where(incentives_range == 15.0)[0]
            # if incentive_index_15pct.size > 0:
            #     participation_at_15pct = pd.DataFrame({
            #         'Dauer (h)': Y_durations[:, incentive_index_15pct[0]],
            #         'Teilnahmequote (%) bei 15% Anreiz': Z_participation_rate[:, incentive_index_15pct[0]]
            #     })
            #     print(f"\nTeilnahmequoten für {target_appliance_surface} bei 15% Anreiz:")
            #     # display(participation_at_15pct)
            #     print(participation_at_15pct)

        else:
            print(f"Keine validen Daten für den Plot der Teilnahmequote für {target_appliance_surface} vorhanden.")
else:
    print("FEHLER: master_df_f1 ist nicht verfügbar oder `get_flexibility_potential` nicht definiert.")

print("\n--- Abschnitt 4.2: Modellbasiertes Flexibilitätspotenzial (Beispiel Geschirrspüler) Abgeschlossen ---")

Beginne modellbasierte Analyse des Flexibilitätspotenzials (basierend auf master_df_f1)
Verwendete Anreizstufen (Kompensation %): [ 0.  5. 10. 15. 20. 25. 30. 35. 40. 45. 50.]
Verwendete Event-Dauern für die Simulation (basierend auf Q9, max 18.0h): [ 1.5  4.5  9.  18. ] Stunden

Generiere Daten für 3D-Plot für Zielgerät: Geschirrspüler...
Verwendetes repräsentatives Profil für 'Geschirrspüler': Konstant 1.2 kW über 18.0h.
  Bearbeite Dauer: 1.5h für Geschirrspüler...

---- simulate_respondent_level_load_shift GESTARTET ----
[INPUT] df_respondent_flexibility Shape: (155, 19)
[INPUT] df_average_load_profiles Shape: (72, 1), Index: 2024-01-01 12:00:00 bis 2024-01-02 05:45:00
[INPUT] Event Parameter: {'start_time': datetime.datetime(2024, 1, 1, 12, 0), 'end_time': datetime.datetime(2024, 1, 1, 13, 30), 'required_duration_hours': np.float64(1.5), 'incentive_percentage': np.float64(0.0)}
[INPUT] Simulationsannahmen: {'reality_discount_factor': 0.7, 'payback_model': {'type': 'none'}}

[REDUK

/var/folders/94/qb4h3ndj5kbdnm3m3qh9cpv80000gn/T/ipykernel_60181/595351301.py:62: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.




--- Abschnitt 4.2: Modellbasiertes Flexibilitätspotenzial (Beispiel Geschirrspüler) Abgeschlossen ---


In [112]:
# Abschnitt 5: Einflussfaktoren auf die Höhe der Kompensationsforderungen
## 5.1 Einfluss soziodemografischer Faktoren auf die Kompensationsforderungen
# Stelle sicher, dass df_compens_demand_stated aus Abschnitt 4.1 existiert und nicht leer ist.
# df_respondents_unique enthält die soziodemografischen Daten pro Respondent.
# Wir müssen df_compens_demand_stated (mit gerätespezifischen Kompensationsforderungen)
# mit den soziodemografischen Daten aus df_respondents_unique mergen,
# falls die soziodem. Spalten nicht bereits durch frühere Merges in df_compens_demand_stated sind.

# Überprüfen, ob die soziodemografischen Spalten bereits in df_compens_demand_stated sind.
# master_df_f1 hatte bereits alles gemerged. df_compens_demand_stated ist ein Subset davon.
# Daher sollten Spalten wie 'age', 'q13_income', 'q14_education' bereits vorhanden sein.

if 'df_compens_demand_stated' in locals() and not df_compens_demand_stated.empty:
    print(f"Beginne Analyse des Einflusses soziodemografischer Faktoren auf Kompensationsforderungen (N valid compens. demands={len(df_compens_demand_stated)}).")
    
    # --- Einfluss Einkommen (Q13) ---
    if 'q13_income' in df_compens_demand_stated.columns:
        print("\n--- Einfluss Einkommen (Q13) auf Kompensationsforderung ---")
        # Definiere die korrekte Reihenfolge der Einkommenskategorien für den Plot
        income_categories_order_q13 = [
            "Unter 3.000 CHF", "3.000 - 5.000 CHF", "5.001 - 7.000 CHF",
            "7.001 - 10.000 CHF", "Über 10.000 CHF", "Keine Angabe"
        ]
        
        # Erstelle eine kategorische Spalte mit der definierten Ordnung
        # df_compens_demand_stated['q13_income_cat_ordered'] = pd.Categorical(
        #    df_compens_demand_stated['q13_income'], categories=income_categories_order_q13, ordered=True
        # )

        fig_income_compens = px.box(df_compens_demand_stated.dropna(subset=['q13_income', 'incentive_pct_required_num']),
                                     x='q13_income', y='incentive_pct_required_num',
                                     title='Kompensationsforderung nach Haushaltseinkommen (Q13)',
                                     labels={'q13_income': 'Monatl. Haushaltsnettoeinkommen',
                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'},
                                     category_orders={'q13_income': income_categories_order_q13}) # Sortierung direkt im Plot
        fig_income_compens.update_xaxes(tickangle=30)
        fig_income_compens.show()
        
        print("Deskriptive Statistik der Kompensationsforderung nach Einkommen:")
        stats_income_compens = df_compens_demand_stated.groupby('q13_income')['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2)
        # display(stats_income_compens.reindex(income_categories_order_q13)) # Sortieren der Tabelle
        print(stats_income_compens.reindex(income_categories_order_q13))

    else:
        print("WARNUNG: Spalte 'q13_income' nicht in df_compens_demand_stated gefunden.")

    # --- Einfluss Alter (Q1) ---
    # 'age' sollte bereits numerisch und in df_compens_demand_stated vorhanden sein.
    if 'age' in df_compens_demand_stated.columns:
        print("\n--- Einfluss Alter (Q1) auf Kompensationsforderung ---")
        # Altersgruppen bilden für die Visualisierung (wie in deinem ursprünglichen Notebook)
        age_bins = [0, 25, 35, 45, 55, 65, df_compens_demand_stated['age'].max() + 1] # Max Alter dynamisch
        age_labels = ["<25", "25-35", "36-45", "46-55", "56-65", "65+"]
        # Stelle sicher, dass age_labels zur Anzahl der Bins passt (-1)
        if len(age_labels) != len(age_bins) - 1:
            print(f"WARNUNG: Anzahl der age_labels ({len(age_labels)}) passt nicht zur Anzahl der age_bins ({len(age_bins)}). Passe Bins/Labels an.")
            # Fallback oder Anpassung hier, falls nötig
        
        df_compens_demand_stated['age_group'] = pd.cut(
            df_compens_demand_stated['age'], bins=age_bins, labels=age_labels, right=False, include_lowest=True
        )
        
        fig_age_compens = px.box(df_compens_demand_stated.dropna(subset=['age_group', 'incentive_pct_required_num']),
                                 x='age_group', y='incentive_pct_required_num',
                                 title='Kompensationsforderung nach Altersgruppe (Q1)',
                                 labels={'age_group': 'Altersgruppe',
                                         'incentive_pct_required_num': 'Geforderte Kompensation (%)'},
                                 category_orders={'age_group': age_labels}) # Sortierung für den Plot
        fig_age_compens.show()
        
        print("Deskriptive Statistik der Kompensationsforderung nach Altersgruppe:")
        stats_age_compens = df_compens_demand_stated.groupby('age_group', observed=False)['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2)
        # display(stats_age_compens)
        print(stats_age_compens)
    else:
        print("WARNUNG: Spalte 'age' nicht in df_compens_demand_stated gefunden.")
        
    # --- Einfluss Bildungsabschluss (Q14) ---
    if 'q14_education' in df_compens_demand_stated.columns:
        print("\n--- Einfluss Bildungsabschluss (Q14) auf Kompensationsforderung ---")
        # Definiere die korrekte Reihenfolge der Bildungskategorien für den Plot
        education_categories_order_q14 = [ # Passe diese an deine exakten Kategorien an!
            "Keine Schulbildung", "Grundschule", "Sekundarschule/Realschule",
            "Berufsausbildung/Lehre/Maturität", "Fachhochschule/Bachelor",
            "Universität/Master", "Promotion oder höher", "Keine Angabe"
        ]
        
        fig_edu_compens = px.box(df_compens_demand_stated.dropna(subset=['q14_education', 'incentive_pct_required_num']),
                                     x='q14_education', y='incentive_pct_required_num',
                                     title='Kompensationsforderung nach höchstem Bildungsabschluss (Q14)',
                                     labels={'q14_education': 'Bildungsabschluss',
                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'},
                                     category_orders={'q14_education': education_categories_order_q14}) # Sortierung
        fig_edu_compens.update_xaxes(tickangle=30)
        fig_edu_compens.show()
        
        print("Deskriptive Statistik der Kompensationsforderung nach Bildungsabschluss:")
        stats_edu_compens = df_compens_demand_stated.groupby('q14_education')['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2)
        # display(stats_edu_compens.reindex(education_categories_order_q14, fill_value=0)) # Sortieren und fehlende mit 0 füllen
        # Sicherer reindexen, nur mit existierenden Kategorien:
        existing_edu_cats = [cat for cat in education_categories_order_q14 if cat in stats_edu_compens.index]
        print(stats_edu_compens.reindex(existing_edu_cats))

    else:
        print("WARNUNG: Spalte 'q14_education' nicht in df_compens_demand_stated gefunden.")
else:
    print("FEHLER: df_compens_demand_stated ist nicht verfügbar oder leer. Analyse der Einflussfaktoren kann nicht durchgeführt werden.")

print("\n--- Abschnitt 5.1: Einfluss soziodemografischer Faktoren Abgeschlossen ---")

Beginne Analyse des Einflusses soziodemografischer Faktoren auf Kompensationsforderungen (N valid compens. demands=187).

--- Einfluss Einkommen (Q13) auf Kompensationsforderung ---


Deskriptive Statistik der Kompensationsforderung nach Einkommen:
                     mean  median    std  count
q13_income                                     
Unter 3.000 CHF     14.74    10.0  13.51     19
3.000 - 5.000 CHF   21.64    10.0  25.50     28
5.001 - 7.000 CHF   33.55    25.0  32.64     31
7.001 - 10.000 CHF  22.18    20.0  16.57     39
Über 10.000 CHF     21.27    20.0  15.37     45
Keine Angabe        20.32    12.0  17.16     25

--- Einfluss Alter (Q1) auf Kompensationsforderung ---


Deskriptive Statistik der Kompensationsforderung nach Altersgruppe:
            mean  median    std  count
age_group                             
<25        27.34    20.0  26.71     61
25-35      22.69    10.0  22.34     54
36-45      19.96    20.0  10.94     25
46-55      23.57    20.0  18.52     14
56-65      19.48    10.0  16.53     21
65+        10.00    10.0   5.35      8

--- Einfluss Bildungsabschluss (Q14) auf Kompensationsforderung ---


Deskriptive Statistik der Kompensationsforderung nach Bildungsabschluss:
                                   mean  median    std  count
q14_education                                                
Sekundarschule/Realschule         18.00    10.0  18.23      5
Berufsausbildung/Lehre/Maturität  24.53    17.5  26.01     90
Fachhochschule/Bachelor           19.60    20.0  13.22     45
Universität/Master                23.80    17.5  18.80     44
Promotion oder höher               9.67    10.0   0.58      3

--- Abschnitt 5.1: Einfluss soziodemografischer Faktoren Abgeschlossen ---


In [113]:
# Stelle sicher, dass df_respondents_unique aus Abschnitt 1 existiert und nicht leer ist.
if 'df_compens_demand_stated' in locals() and not df_compens_demand_stated.empty:
    print(f"Beginne Analyse des Einflusses von Einstellungs-/Kontextfaktoren auf Kompensationsforderungen (N valid compens. demands={len(df_compens_demand_stated)}).")

    # --- Einfluss Geräte-Wichtigkeit (Q8) ---
    # 'importance_rating' sollte bereits numerisch sein (Skala 1-5).
    if 'importance_rating' in df_compens_demand_stated.columns:
        print("\n--- Einfluss Geräte-Wichtigkeit (Q8) auf Kompensationsforderung ---")
        
        # Stelle sicher, dass importance_rating numerisch ist und keine unerwarteten Strings enthält
        df_compens_demand_stated['importance_rating_num_q8'] = pd.to_numeric(df_compens_demand_stated['importance_rating'], errors='coerce')
        
        # Entferne Zeilen, wo importance_rating_num_q8 NaN ist für diese spezifische Analyse
        df_q8_analysis = df_compens_demand_stated.dropna(subset=['importance_rating_num_q8', 'incentive_pct_required_num'])

        if not df_q8_analysis.empty:
            fig_importance_compens = px.box(df_q8_analysis,
                                            x='importance_rating_num_q8', y='incentive_pct_required_num',
                                            title='Kompensationsforderung nach GerWichtigkeit (Q8)',
                                            labels={'importance_rating_num_q8': 'Wichtigkeit des Geräts (1=sehr unwichtig, 5=sehr wichtig)',
                                                    'incentive_pct_required_num': 'Geforderte Kompensation (%)'},
                                            category_orders={'importance_rating_num_q8': sorted(df_q8_analysis['importance_rating_num_q8'].unique())})
            fig_importance_compens.show()
            
            print("Deskriptive Statistik der Kompensationsforderung nach Gerätewichtigkeit (Q8):")
            stats_importance_compens = df_q8_analysis.groupby('importance_rating_num_q8')['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2)
            # display(stats_importance_compens)
            print(stats_importance_compens)
        else:
            print("Keine validen Daten für die Analyse nach Gerätewichtigkeit vorhanden (nach dropna).")
    else:
        print("WARNUNG: Spalte 'importance_rating' nicht in df_compens_demand_stated gefunden.")


if 'df_respondents_unique' in locals() and not df_respondents_unique.empty:
    print(f"Beginne Analyse der Akzeptanz von Enabling-Technologien (Q11, Q12) basierend auf df_respondents_unique (N={len(df_respondents_unique)}).")

    # --- Akzeptanz Benachrichtigungen (Q11) ---
    if 'q11_notify' in df_respondents_unique.columns:
        print("\n--- Akzeptanz von Benachrichtigungen (Q11) ---")
        q11_counts = df_respondents_unique['q11_notify'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        q11_counts.rename(index={np.nan: 'Keine Angabe Q11'}, inplace=True)
        print("Verteilung der Antworten zu Q11 (Benachrichtigungen):")
        # display(q11_counts.to_frame(name='Anteil (%)'))
        print(q11_counts.to_frame(name='Anteil (%)'))
        
        fig_q11_acceptance = px.bar(q11_counts.reset_index(), x='q11_notify', y='proportion',
                                   title='Q11: Bereitschaft, Benachrichtigungen zu erhalten',
                                   labels={'q11_notify': 'Antwort', 'proportion': 'Anteil (%)'},
                                   text_auto=True)
        fig_q11_acceptance.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_q11_acceptance.show()
    else:
        print("WARNUNG: Spalte 'q11_notify' nicht in df_respondents_unique gefunden.")

    # --- Akzeptanz Smart Plugs (Q12) ---
    if 'q12_smartplug' in df_respondents_unique.columns:
        print("\n--- Akzeptanz von Smart Plugs (Q12) ---")
        q12_counts = df_respondents_unique['q12_smartplug'].value_counts(normalize=True, dropna=False).mul(100).round(1)
        q12_counts.rename(index={np.nan: 'Keine Angabe Q12'}, inplace=True)
        print("Verteilung der Antworten zu Q12 (Smart Plugs):")
        # display(q12_counts.to_frame(name='Anteil (%)'))
        print(q12_counts.to_frame(name='Anteil (%)'))

        fig_q12_acceptance = px.bar(q12_counts.reset_index(), x='q12_smartplug', y='proportion',
                                   title='Q12: Bereitschaft zur Installation von Smart Plugs',
                                   labels={'q12_smartplug': 'Antwort', 'proportion': 'Anteil (%)'},
                                   text_auto=True)
        fig_q12_acceptance.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_q12_acceptance.show()
    else:
        print("WARNUNG: Spalte 'q12_smartplug' nicht in df_respondents_unique gefunden.")

    # --- Zusammenhang Q11 (Benachrichtigungen) und Q12 (Smart Plugs) ---
    if 'q11_notify' in df_respondents_unique.columns and 'q12_smartplug' in df_respondents_unique.columns:
        print("\n--- Zusammenhang zwischen Smart Plug Akzeptanz (Q12) und Benachrichtigungswunsch (Q11) ---")
        
        # Ersetze NaN durch "Keine Angabe" für die Kreuztabelle, um sie sichtbar zu machen
        q11_for_crosstab = df_respondents_unique['q11_notify'].fillna('Keine Angabe Q11')
        q12_for_crosstab = df_respondents_unique['q12_smartplug'].fillna('Keine Angabe Q12')
        
        # Kreuztabelle mit absoluten Häufigkeiten
        crosstab_q11_q12_abs = pd.crosstab(q12_for_crosstab, q11_for_crosstab, margins=True, margins_name="Gesamt")
        print("Kreuztabelle (absolute Häufigkeiten): Q12 (Smart Plug) vs. Q11 (Benachrichtigung)")
        # display(crosstab_q11_q12_abs)
        print(crosstab_q11_q12_abs)

        # Kreuztabelle mit prozentualen Anteilen (bezogen auf die Gesamtsumme aller Befragten)
        crosstab_q11_q12_pct = pd.crosstab(q12_for_crosstab, q11_for_crosstab, normalize='all').mul(100).round(1)
        print("\nKreuztabelle (prozentuale Anteile der Gesamtstichprobe N=155): Q12 (Smart Plug) vs. Q11 (Benachrichtigung)")
        # display(crosstab_q11_q12_pct)
        print(crosstab_q11_q12_pct)
        
        # Visualisierung der Kreuztabelle (z.B. gruppiertes oder gestapeltes Balkendiagramm)
        # Umformatieren für Plotly Express
        crosstab_plot_df = crosstab_q11_q12_pct.stack().rename("percentage").reset_index()
        
        fig_crosstab_q11_q12 = px.bar(crosstab_plot_df, x='q12_smartplug', y='percentage', color='q11_notify',
                                     barmode='group',
                                     title='Kombinierte Akzeptanz: Smart Plug (Q12) und Benachrichtigung (Q11)',
                                     labels={'q12_smartplug': 'Smart Plug Akzeptanz (Q12)',
                                             'percentage': 'Anteil an Gesamtstichprobe (%)',
                                             'q11_notify': 'Benachrichtigung erwünscht (Q11)'},
                                     text_auto=True)
        fig_crosstab_q11_q12.update_traces(texttemplate='%{y:.1f}%', textposition='outside', cliponaxis=False)
        fig_crosstab_q11_q12.show()
    else:
        print("WARNUNG: Spalten 'q11_notify' oder 'q12_smartplug' nicht für Kreuztabellenanalyse gefunden.")
else:
    print("FEHLER: df_respondents_unique ist nicht verfügbar oder leer.")

# print("\n--- (Neuer Teil von) Abschnitt 5.2: Akzeptanz Enabling-Technologien Abgeschlossen ---") 
# Die Gesamtüberschrift bleibt Abschnitt 5.2

Beginne Analyse des Einflusses von Einstellungs-/Kontextfaktoren auf Kompensationsforderungen (N valid compens. demands=187).

--- Einfluss Geräte-Wichtigkeit (Q8) auf Kompensationsforderung ---


Deskriptive Statistik der Kompensationsforderung nach Gerätewichtigkeit (Q8):
                           mean  median    std  count
importance_rating_num_q8                             
1.0                       19.86    10.0  22.16     21
2.0                       22.03    20.0  20.29     29
3.0                       19.47    15.0  17.52     45
4.0                       25.97    15.0  25.58     38
5.0                       24.76    20.0  21.93     54
Beginne Analyse der Akzeptanz von Enabling-Technologien (Q11, Q12) basierend auf df_respondents_unique (N=155).

--- Akzeptanz von Benachrichtigungen (Q11) ---
Verteilung der Antworten zu Q11 (Benachrichtigungen):
            Anteil (%)
q11_notify            
Ja                87.7
Nein              12.3



--- Akzeptanz von Smart Plugs (Q12) ---
Verteilung der Antworten zu Q12 (Smart Plugs):
               Anteil (%)
q12_smartplug            
Ja                   82.6
Nein                 17.4



--- Zusammenhang zwischen Smart Plug Akzeptanz (Q12) und Benachrichtigungswunsch (Q11) ---
Kreuztabelle (absolute Häufigkeiten): Q12 (Smart Plug) vs. Q11 (Benachrichtigung)
q11_notify      Ja  Nein  Gesamt
q12_smartplug                   
Ja             119     9     128
Nein            17    10      27
Gesamt         136    19     155

Kreuztabelle (prozentuale Anteile der Gesamtstichprobe N=155): Q12 (Smart Plug) vs. Q11 (Benachrichtigung)
q11_notify       Ja  Nein
q12_smartplug            
Ja             76.8   5.8
Nein           11.0   6.5


In [114]:
# Abschnitt 6: Zusammenfassung der Kernergebnisse für Forschungsfrage F1
# Abschnitt 7: Export von ausgewählten Ergebnissen

# Erstelle den outputs-Ordner, falls er nicht existiert
# Wir erstellen einen spezifischen Unterordner für diese Analyse, um Ordnung zu halten.
output_dir_f1_v2 = PROJECT_ROOT_NB / "scripts" / "F1_Analyse_Kompensationsforderungen" / "outputs_f1_v2"
os.makedirs(output_dir_f1_v2, exist_ok=True)
print(f"Ausgaben werden in '{output_dir_f1_v2}' gespeichert.")

# --- Tabellen exportieren (als CSV) ---
print("\nExportiere Tabellen...")
try:
    # Deskriptive Statistik der Stichprobe (Beispiel: Einkommen)
    # Du müsstest für jede Tabelle aus Abschnitt 1, die du speichern willst, den Code wiederholen.
    # Annahme: stats_income_compens (und ähnliche für Alter, Bildung etc.) sind noch im Speicher.
    # Falls nicht, hier Code aus Abs. 1 wiederholen oder sicherstellen, dass die Zellen oben ausgeführt wurden.
    if 'stats_income_compens' in locals():
        stats_income_compens.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_komp_nach_einkommen.csv")
        print("  Tabelle 'komp_nach_einkommen.csv' gespeichert.")
    if 'stats_age_compens' in locals():
        stats_age_compens.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_komp_nach_alter.csv")
        print("  Tabelle 'komp_nach_alter.csv' gespeichert.")
    if 'stats_edu_compens' in locals():
        stats_edu_compens.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_komp_nach_bildung.csv")
        print("  Tabelle 'komp_nach_bildung.csv' gespeichert.")

    # Gerätewichtigkeit (Q8)
    if 'q8_stats_per_device' in locals(): # Aus Abschnitt 3.1
        q8_stats_per_device.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_q8_wichtigkeit_pro_geraet.csv")
        print("  Tabelle 'q8_wichtigkeit_pro_geraet.csv' gespeichert.")
        
    # Akzeptierte Nichtnutzungsdauer (Q9) pro Gerät
    if 'q9_duration_per_device_table' in locals(): # Aus Abschnitt 3.2
        q9_duration_per_device_table.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_q9_dauer_pro_geraet.csv")
        print("  Tabelle 'q9_dauer_pro_geraet.csv' gespeichert.")

    # Teilnahmebereitschaft (Q10) pro Gerät
    if 'q10_by_device_table' in locals(): # Aus Abschnitt 3.3.2
        q10_by_device_table.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_q10_bereitschaft_pro_geraet.csv")
        print("  Tabelle 'q10_bereitschaft_pro_geraet.csv' gespeichert.")

    # Explizit genannte Kompensationsforderungen pro Gerät (Median, Mean etc.)
    if 'stated_compens_stats_per_device' in locals(): # Aus Abschnitt 4.1
        stated_compens_stats_per_device.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_explizite_komp_pro_geraet.csv")
        print("  Tabelle 'explizite_komp_pro_geraet.csv' gespeichert.")

    # Einstellungen/Kontextfaktoren (Q8, Q11, Q12)
    if 'stats_importance_compens' in locals(): # Aus Abschnitt 5.2.1
        stats_importance_compens.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_komp_nach_q8wichtigkeit.csv")
        print("  Tabelle 'komp_nach_q8wichtigkeit.csv' gespeichert.")
    if 'crosstab_q11_q12_abs' in locals(): # Aus Abschnitt 5.2.2
        crosstab_q11_q12_abs.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_kreuztabelle_q11_q12_abs.csv")
        print("  Tabelle 'kreuztabelle_q11_q12_abs.csv' gespeichert.")
    if 'crosstab_q11_q12_pct' in locals(): # Aus Abschnitt 5.2.2
        crosstab_q11_q12_pct.to_csv(output_dir_f1_v2 / "f1_v2_tabelle_kreuztabelle_q11_q12_prozent.csv")
        print("  Tabelle 'kreuztabelle_q11_q12_prozent.csv' gespeichert.")
        
except Exception as e:
    print(f"Fehler beim Speichern einer oder mehrerer Tabellen: {e}")

# --- Grafiken exportieren (als HTML und optional PNG/SVG) ---
# Du musst sicherstellen, dass die Plotly-Figuren-Variablen noch im Speicher sind.
# Ihre Namen sind z.B. fig_q8_importance, fig_q9_duration_device, fig_q10_device, fig_median_compens_device, etc.
# Und die aus Abschnitt 5.1 (fig_income_compens, fig_age_compens, fig_edu_compens)
# Und die aus Abschnitt 5.2 (fig_importance_compens, fig_q11_acceptance, fig_q12_acceptance, fig_crosstab_q11_q12)

print("\nExportiere Grafiken...")
figures_to_export = {
    # Aus Abschnitt 1 (Stichprobenbeschreibung) - falls du sie exportieren willst
    # "01_hist_alter": fig_age_hist, # Beispiel, falls fig_age_hist noch existiert
    # "02_bar_geschlecht": fig_gender,
    # ... (weitere aus Abschnitt 1)

    # Aus Abschnitt 3
    "fig_ab02_q8_wichtigkeit": 'fig_q8_importance' if 'fig_q8_importance' in locals() else None, # Deine Abbildung 2
    "fig_ab03_q9_dauer_pro_geraet": 'fig_q9_duration_device' if 'fig_q9_duration_device' in locals() else None, # Deine Abbildung 3
    "fig_ab04_q10_bereitschaft_pro_geraet": 'fig_q10_device' if 'fig_q10_device' in locals() else None, # Deine Abbildung 4
    
    # Aus Abschnitt 4.1
    "fig_ab05_median_komp_pro_geraet": 'fig_median_compens_device' if 'fig_median_compens_device' in locals() else None, # Deine Abbildung 5
    
    # Aus Abschnitt 5.1
    "fig_ab06_komp_nach_einkommen": 'fig_income_compens' if 'fig_income_compens' in locals() else None, # Deine Abbildung 6
    "fig_ab07_komp_nach_alter": 'fig_age_compens' if 'fig_age_compens' in locals() else None, # Deine Abbildung 7
    "fig_ab08_komp_nach_bildung": 'fig_edu_compens' if 'fig_edu_compens' in locals() else None, # Deine Abbildung 8

    # Aus Abschnitt 5.2
    "fig_ab09_komp_nach_q8wichtigkeit": 'fig_importance_compens' if 'fig_importance_compens' in locals() else None, # Deine Abbildung 9
    "fig_ab10_akzeptanz_q11_benachrichtigung": 'fig_q11_acceptance' if 'fig_q11_acceptance' in locals() else None, # Deine Abbildung 10
    "fig_ab11_akzeptanz_q12_smartplug": 'fig_q12_acceptance' if 'fig_q12_acceptance' in locals() else None, # Deine Abbildung 11
    "fig_ab12_kreuztabelle_q11_q12": 'fig_crosstab_q11_q12' if 'fig_crosstab_q11_q12' in locals() else None # Deine Abbildung 12
}

for fig_name_prefix, fig_variable_name_str in figures_to_export.items():
    if fig_variable_name_str and fig_variable_name_str in locals():
        fig_object = locals()[fig_variable_name_str]
        try:
            # HTML Export (interaktiv)
            html_path = output_dir_f1_v2 / f"{fig_name_prefix}.html"
            fig_object.write_html(html_path)
            print(f"  Grafik '{html_path.name}' gespeichert.")
            
            # Optional: PNG/SVG Export (statisch) - benötigt kaleido: pip install -U kaleido
            # png_path = output_dir_f1_v2 / f"{fig_name_prefix}.png"
            # fig_object.write_image(png_path, scale=2) # scale für höhere Auflösung
            # print(f"  Grafik '{png_path.name}' gespeichert.")
            
        except Exception as e:
            print(f"    Fehler beim Speichern von Grafik '{fig_name_prefix}': {e}")
    elif fig_variable_name_str:
        print(f"WARNUNG: Grafik-Variable '{fig_variable_name_str}' für '{fig_name_prefix}' nicht im Speicher gefunden. Export übersprungen.")

print("\n--- Abschnitt 7: Export von Ergebnissen Abgeschlossen ---")

Ausgaben werden in '/Users/jonathan/Documents/GitHub/PowerE/scripts/F1_Analyse_Kompensationsforderungen/outputs_f1_v2' gespeichert.

Exportiere Tabellen...
  Tabelle 'komp_nach_einkommen.csv' gespeichert.
  Tabelle 'komp_nach_alter.csv' gespeichert.
  Tabelle 'komp_nach_bildung.csv' gespeichert.
  Tabelle 'q8_wichtigkeit_pro_geraet.csv' gespeichert.
  Tabelle 'q9_dauer_pro_geraet.csv' gespeichert.
  Tabelle 'q10_bereitschaft_pro_geraet.csv' gespeichert.
  Tabelle 'explizite_komp_pro_geraet.csv' gespeichert.
  Tabelle 'komp_nach_q8wichtigkeit.csv' gespeichert.
  Tabelle 'kreuztabelle_q11_q12_abs.csv' gespeichert.
  Tabelle 'kreuztabelle_q11_q12_prozent.csv' gespeichert.

Exportiere Grafiken...
  Grafik 'fig_ab02_q8_wichtigkeit.html' gespeichert.
  Grafik 'fig_ab03_q9_dauer_pro_geraet.html' gespeichert.
  Grafik 'fig_ab04_q10_bereitschaft_pro_geraet.html' gespeichert.
  Grafik 'fig_ab05_median_komp_pro_geraet.html' gespeichert.
  Grafik 'fig_ab06_komp_nach_einkommen.html' gespeichert.
  